In [1]:
import sys
sys.executable
import os
os.getcwd()
sys.path.append('devel/lib/python3/dist-packages/theconstruct_msgs/msg/')
sys.path.append('/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/src')
sys.path.append('/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/src/moveo_training/scripts')
sys.path.append('/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/devel/lib/python3/dist-packages/openai_ros/msg')

In [2]:
import base64
import imageio
import IPython
import matplotlib.pyplot as plt
import os
import reverb
import tempfile
import PIL.Image

import tensorflow as tf
import moveo_push
from tf_agents.environments import suite_gym
from tf_agents.agents.ddpg import critic_network
from tf_agents.agents.sac import sac_agent
from tf_agents.agents.sac import tanh_normal_projection_network
from tf_agents.environments import suite_pybullet
from tf_agents.metrics import py_metrics
from tf_agents.networks import actor_distribution_network
from tf_agents.policies import greedy_policy
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_py_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.train import actor
from tf_agents.train import learner
from tf_agents.train import triggers
from tf_agents.train.utils import spec_utils
from tf_agents.train.utils import strategy_utils
from tf_agents.train.utils import train_utils
import rospy
from gym.envs.registration import register
import gym
tempdir = tempfile.gettempdir()

2021-08-21 13:01:05.996503: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/devel/lib:/opt/ros/noetic/lib:/opt/ros/noetic/lib/x86_64-linux-gnu
2021-08-21 13:01:05.996544: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Failed to import pyassimp, see https://github.com/ros-planning/moveit/issues/86 for more info


In [3]:
tempdir

'/tmp'

In [4]:
num_iterations = 5000 # @param {type:"integer"}

initial_collect_steps = 500 # @param {type:"integer"}
collect_steps_per_iteration = 1 # @param {type:"integer"}
replay_buffer_capacity = 1000 # @param {type:"integer"}

batch_size = 256 # @param {type:"integer"}

critic_learning_rate = 3e-4 # @param {type:"number"}
actor_learning_rate = 3e-4 # @param {type:"number"}
alpha_learning_rate = 3e-4 # @param {type:"number"}
target_update_tau = 0.005 # @param {type:"number"}
target_update_period = 1 # @param {type:"number"}
gamma = 0.99 # @param {type:"number"}
reward_scale_factor = 1.0 # @param {type:"number"}

actor_fc_layer_params = (256, 256)
critic_joint_fc_layer_params = (256, 256)

log_interval = 50# @param {type:"integer"}

num_eval_episodes = 10 # @param {type:"integer"}
eval_interval = 100 # @param {type:"integer"}

policy_save_interval = 1000 # @param {type:"integer"}

In [5]:
# env_name = 'MoveoIK-v0'
# if env_name in gym.envs.registry.env_specs:
#     del gym.envs.registry.env_specs[env_name]



In [6]:
# env_name = 'MoveoIK-v0'
# if env_name in gym.envs.registry.env_specs:
#     del gym.envs.registry.env_specs[env_name]

# register(
#         id=env_name ,
#         entry_point='moveo_inverse_kinematic_2:MoveoIKEnv',
#         max_episode_steps=max_episode_steps,
#     )

In [7]:
rospy.init_node("train_moveo_push")
env_name = 'MoveoPush-v0' # @param {type:"string"}
env = suite_gym.load(env_name)
moveo_push

###### Setting Group Moveit with 30 seconds wait...


[ERROR] [1629543670.091612, 587462.461000]: NOT Initialising Simulation Physics Parameters
[WARN] [1629543670.105613, 587464.301000]: Start Init ControllersConnection
[WARN] [1629543670.107555, 587464.381000]: END Init ControllersConnection
/home/nils/.local/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [8]:
env.reset()

TimeStep(
{'discount': array(1., dtype=float32),
 'observation': array([0.3, 0.3, 0.3], dtype=float32),
 'reward': array(0., dtype=float32),
 'step_type': array(0, dtype=int32)})

In [9]:
print('Observation Spec:')
print(env.time_step_spec().observation)
print('Action Spec:')
print(env.action_spec())

Observation Spec:
BoundedArraySpec(shape=(3,), dtype=dtype('float32'), name='observation', minimum=-1.0, maximum=1.0)
Action Spec:
BoundedArraySpec(shape=(5,), dtype=dtype('float32'), name='action', minimum=-3.140000104904175, maximum=3.140000104904175)


In [10]:
collect_env = suite_gym.load(env_name)
eval_env = suite_gym.load(env_name)

###### Setting Group Moveit with 30 seconds wait...
###### Setting Group Moveit with 30 seconds wait...


[ERROR] [1629543671.297824, 587541.871000]: NOT Initialising Simulation Physics Parameters
[WARN] [1629543671.316260, 587544.181000]: Start Init ControllersConnection
[WARN] [1629543671.317721, 587544.191000]: END Init ControllersConnection
[ERROR] [1629543671.431914, 587553.331000]: NOT Initialising Simulation Physics Parameters
[WARN] [1629543671.451190, 587554.401000]: Start Init ControllersConnection
[WARN] [1629543671.452949, 587554.401000]: END Init ControllersConnection


In [11]:
use_gpu = True
strategy = strategy_utils.get_strategy(tpu=False, use_gpu=use_gpu)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


2021-08-21 13:01:11.534775: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-08-21 13:01:11.565962: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-21 13:01:11.566533: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 970 computeCapability: 5.2
coreClock: 1.2155GHz coreCount: 13 deviceMemorySize: 3.94GiB deviceMemoryBandwidth: 208.91GiB/s
2021-08-21 13:01:11.566837: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/devel/lib:/opt/ros/noetic/lib:/opt/ros

In [12]:
observation_spec, action_spec, time_step_spec = (
      spec_utils.get_tensor_specs(collect_env))

In [13]:
with strategy.scope():
  critic_net = critic_network.CriticNetwork(
        (observation_spec, action_spec),
        observation_fc_layer_params=None,
        action_fc_layer_params=None,
        joint_fc_layer_params=critic_joint_fc_layer_params,
        kernel_initializer='glorot_uniform',
        last_kernel_initializer='glorot_uniform')

In [14]:
with strategy.scope():
  actor_net = actor_distribution_network.ActorDistributionNetwork(
      observation_spec,
      action_spec,
      fc_layer_params=actor_fc_layer_params,
      continuous_projection_net=(
          tanh_normal_projection_network.TanhNormalProjectionNetwork))

In [15]:
with strategy.scope():
  train_step = train_utils.create_train_step()

  tf_agent = sac_agent.SacAgent(
        time_step_spec,
        action_spec,
        actor_network=actor_net,
        critic_network=critic_net,
        actor_optimizer=tf.compat.v1.train.AdamOptimizer(
            learning_rate=actor_learning_rate),
        critic_optimizer=tf.compat.v1.train.AdamOptimizer(
            learning_rate=critic_learning_rate),
        alpha_optimizer=tf.compat.v1.train.AdamOptimizer(
            learning_rate=alpha_learning_rate),
        target_update_tau=target_update_tau,
        target_update_period=target_update_period,
        td_errors_loss_fn=tf.math.squared_difference,
        gamma=gamma,
        reward_scale_factor=reward_scale_factor,
        train_step_counter=train_step)

  tf_agent.initialize()

In [16]:
rate_limiter=reverb.rate_limiters.SampleToInsertRatio(samples_per_insert=3.0, min_size_to_sample=3, error_buffer=3.0)

In [17]:
table_name = 'uniform_table'
table = reverb.Table(
    table_name,
    max_size=replay_buffer_capacity,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1))

reverb_server = reverb.Server([table])

[reverb/cc/platform/tfrecord_checkpointer.cc:150]  Initializing TFRecordCheckpointer in /tmp/tmpk1ez9lgp.
[reverb/cc/platform/tfrecord_checkpointer.cc:378] Loading latest checkpoint from /tmp/tmpk1ez9lgp
[reverb/cc/platform/default/server.cc:54] Started replay server on port 21388


In [18]:
reverb_replay = reverb_replay_buffer.ReverbReplayBuffer(
    tf_agent.collect_data_spec,
    sequence_length=2,
    table_name=table_name,
    local_server=reverb_server)

In [19]:
dataset = reverb_replay.as_dataset(
      sample_batch_size=batch_size, num_steps=2).prefetch(50)
experience_dataset_fn = lambda: dataset

In [20]:
tf_eval_policy = tf_agent.policy
eval_policy = py_tf_eager_policy.PyTFEagerPolicy(
  tf_eval_policy, use_tf_function=True)

In [21]:
tf_collect_policy = tf_agent.collect_policy
collect_policy = py_tf_eager_policy.PyTFEagerPolicy(
  tf_collect_policy, use_tf_function=True)

In [22]:
random_policy = random_py_policy.RandomPyPolicy(
  collect_env.time_step_spec(), collect_env.action_spec())

In [23]:
rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
  reverb_replay.py_client,
  table_name,
  sequence_length=2,
  stride_length=1)

In [24]:
initial_collect_actor = actor.Actor(
  collect_env,
  random_policy,
  train_step,
  steps_per_run=initial_collect_steps,
  observers=[rb_observer])
initial_collect_actor.run()

[ERROR] [1629543675.384781539, 587889.021000000]: Got a callback on a goalHandle that we're not tracking.                 This is an internal SimpleActionClient/ActionClient bug.                 This could also be a GoalID collision
[ WARN] [1629543686.629164148, 588779.811000000]: Fail: ABORTED: No motion plan found. No execution attempted.
[ WARN] [1629543687.320368011, 588837.911000000]: Fail: ABORTED: No motion plan found. No execution attempted.
[ WARN] [1629543690.170849259, 589093.211000000]: Fail: ABORTED: No motion plan found. No execution attempted.
[ WARN] [1629543696.639217567, 589632.731000000]: Fail: ABORTED: No motion plan found. No execution attempted.
[ WARN] [1629543707.961566874, 590678.031000000]: Fail: ABORTED: No motion plan found. No execution attempted.
[ WARN] [1629543708.524455245, 590730.991000000]: Fail: ABORTED: No motion plan found. No execution attempted.


In [ ]:
env_step_metric = py_metrics.EnvironmentSteps()
collect_actor = actor.Actor(
  collect_env,
  collect_policy,
  train_step,
  steps_per_run=1,
  metrics=actor.collect_metrics(10),
  summary_dir=os.path.join(tempdir, learner.TRAIN_DIR),
  observers=[rb_observer, env_step_metric])

In [ ]:


eval_actor = actor.Actor(
  eval_env,
  eval_policy,
  train_step,
  episodes_per_run=num_eval_episodes,
  metrics=actor.eval_metrics(num_eval_episodes),
  summary_dir=os.path.join(tempdir, 'eval'),
)


In [ ]:
saved_model_dir = os.path.join(tempdir, learner.POLICY_SAVED_MODEL_DIR)

# Triggers to save the agent's policy checkpoints.
learning_triggers = [
    triggers.PolicySavedModelTrigger(
        saved_model_dir,
        tf_agent,
        train_step,
        interval=policy_save_interval),
    triggers.StepPerSecondLogTrigger(train_step, interval=1000),
]
agent_learner = learner.Learner(
  tempdir,
  train_step,
  tf_agent,
  experience_dataset_fn,
  triggers=learning_triggers)


In [ ]:

def get_eval_metrics():
  print("Starte evaluierung")
  eval_actor.run()
  results = {}
  for metric in eval_actor.metrics:
    results[metric.name] = metric.result()
  print("Evaluierung beendete")
  return results

metrics = get_eval_metrics()


In [ ]:


def log_eval_metrics(step, metrics):
  eval_results = (', ').join(
      '{} = {:.6f}'.format(name, result) for name, result in metrics.items())
  print('step = {0}: {1}'.format(step, eval_results))

log_eval_metrics(0, metrics)


In [ ]:
try:
  %%time
except:
  pass


# Reset the train step
tf_agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = get_eval_metrics()["AverageReturn"]
returns = [avg_return]
i =0
for _ in range(num_iterations):
  # Training.
  i +=1
  collect_actor.run()
  loss_info = agent_learner.run(iterations=1)
  
  # Evaluating.
  step = agent_learner.train_step_numpy
  if eval_interval and step % eval_interval == 0:
    metrics = get_eval_metrics()
    log_eval_metrics(step, metrics)
    returns.append(metrics["AverageReturn"])

  if log_interval and step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, loss_info.loss.numpy()))
rb_observer.close()
reverb_server.stop()



In [ ]:

steps = range(0, num_iterations + 1, eval_interval)
plt.plot(steps, returns)
plt.ylabel('Average Return')
plt.xlabel('Step')
plt.ylim()
returns

In [ ]:
get_eval_metrics()

shutdown request: [/train_moveo_her] Reason: new node registered with same name


In [ ]:
plt.scatter(steps,returns)